### Module 10-1 Learning Notebook: Data Preparation for Classification

So far in this course, I've given you only clean, prepared numeric data ready for use in ML models. Would you believe in real life, the data does not always come in this form?<P>
 
In this lesson, we'll start with a slightly unprepared dataset and prepare it to use in ML. Specifically, we'll convert all columns to a numeric form to use in a classification algorithm called Logistic Regression.<P>
    
The dataset we'll work with is very well-known in ML education. It is the characteristics of the passengers on the doomed Titanic:<P>

- https://www.kaggle.com/c/titanic/overview

This dataset contains information about 891 people who were on board the ship when departed on April 15th, 1912. There were not enough lifeboats for everybody so women, children, and the upper-class were prioritized. Using the information about these 891 passengers, the challenge is to build a model to predict which people would survive based on the following fields:

- **Gender** (str) - Gender of the passenger
- **Age** (float) - Age in years
- **Cabin** (str) - Cabin number
- **Embarked** (str) - Name of City of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
    
The target variable:
- **Survived** (int) - Did they survive? (0 = No, 1 = Yes)
    
Our method:
1. Load and investigate the data
2. Convert 'Gender' (currently a string column) to a numeric column
3. Impute missing values in the 'Age' column
4. Convert 'Cabin' from a string to an integer where: 0 = Passenger didn't have a cabin, 1 = Passenger had a cabin
5. Remove 'Embarked' rows with NaN values
6. Convert 'Embarked' column to a set of 3 dummy variable columns (OneHotEncoding)
7. Save your cleaned data to a pickle file on S3

In [2]:
import boto3
import pandas as pd
import numpy as np
from io import BytesIO

### 1. Load and investigate the data
Notice:
- The data types of each of the variables. 'object' means 'string' values in this case.
- There are some missing values in Age, Cabin and Embarked.<BR> 
   Mostly, ML algorithms can't handle missing values
the missing values. 

In [14]:
# Setup boto3
sess = boto3.session.Session()
s3 = sess.client('s3') 
# Define the bucket & file you want to load
source_bucket = 'machinelearning-read-only'
source_key = 'data/titanic_simple.csv'
#
response = s3.get_object(Bucket=source_bucket, Key=source_key)
#
# Load the 'data' part of the response directly into a dataframe
df = pd.read_csv(response.get("Body"))
#
# Let's leave df in place and make a copy of it to work with
df_cleaned = df
df_cleaned.head(6)

,Gender,Age,Cabin,Embarked,Survived
0,male,22.0,NaN,S,0
1,female,38.0,C85,C,1
2,female,26.0,NaN,S,1
3,female,35.0,C123,S,1
4,male,35.0,NaN,S,0
5,male,NaN,NaN,Q,0


In [15]:
# Get the datatypes and any missing values
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gender    891 non-null    object 
 1   Age       714 non-null    float64
 2   Cabin     204 non-null    object 
 3   Embarked  889 non-null    object 
 4   Survived  891 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 34.9+ KB


### 2. Convert 'Gender' (currently a string column) to a numeric column


In [16]:
# Create a dict to map gender to a number
gender_num = {'male': 0, 'female': 1}
# 
# Use the map() function to replace each value in a Series
#    https://sparkbyexamples.com/pandas/pandas-map-function-explained/
df_cleaned['Gender'] = df_cleaned['Gender'].map(gender_num)
df_cleaned.head(6)

,Gender,Age,Cabin,Embarked,Survived
0,0,22.0,NaN,S,0
1,1,38.0,C85,C,1
2,1,26.0,NaN,S,1
3,1,35.0,C123,S,1
4,0,35.0,NaN,S,0
5,0,NaN,NaN,Q,0


### 3. Impute missing values in the 'Age' column

Missing values may take different forms, but here they are of type NaN. <P>

In [17]:
# Show the 6th item
df_cleaned.iloc[5]

Gender        0
Age         NaN
Cabin       NaN
Embarked      Q
Survived      0
Name: 5, dtype: object

#### Impute:
A popular approach for data imputation is to calculate a statistical value for each column (such as a mean) and replace all missing values for that column with the statistic. It is a popular approach because the statistic is easy to calculate using the training dataset and because it often results in good performance.<P>
    
You can read more about it here: https://machinelearningmastery.com/statistical-imputation-for-missing-values-in-machine-learning/

Sklearn gives us an easy way to replace missing values with an imputed value:<P>
https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [18]:
from sklearn.impute import SimpleImputer
# Create the imputer object
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
#
# Convert the 'Age' column to a Numby array
age = df_cleaned['Age'].to_numpy().reshape(-1,1)
#
# fit (or calculate) the imputer object to the array
imp_mean.fit(age)
#
# Calculate all the age values. If they are already a number, they are unchanged.
# If they are NaN, then that is replaced with the average.
age_imputed = imp_mean.transform(age).ravel() # flatten into a simple array
# The 6th item was NaN, now it is the average of the remaining values.
list(age_imputed[0:6])

[22.0, 38.0, 26.0, 35.0, 35.0, 29.69911764705882]

In [19]:
# Add Age back to the dataframe, overwrite existing Age values
df_cleaned['Age'] = age_imputed.tolist() 
# Notice the 6th item is no longer NaN
df_cleaned.head(6)

,Gender,Age,Cabin,Embarked,Survived
0,0,22.000000,NaN,S,0
1,1,38.000000,C85,C,1
2,1,26.000000,NaN,S,1
3,1,35.000000,C123,S,1
4,0,35.000000,NaN,S,0
5,0,29.699118,NaN,Q,0


In [20]:
# Check how we are doing: No missing values in Age
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gender    891 non-null    int64  
 1   Age       891 non-null    float64
 2   Cabin     204 non-null    object 
 3   Embarked  889 non-null    object 
 4   Survived  891 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 34.9+ KB


### 4. Convert 'Cabin' from a string to an integer where: 
0 = Passenger didn't have a cabin, 1 = Passenger had a cabin

In [21]:
# np.where() is like an 'if-then' statement:
# If the Cabin row value is NaN, then assign it a 0, otherwise, assign it a 1
#
# We will assign this to a new column in the dataframe
#
df_cleaned['HasCabin'] = np.where(df_cleaned['Cabin'].isnull(), 0, 1) # Build a new column in the DataFrame
df_cleaned.head(6)

,Gender,Age,Cabin,Embarked,Survived,HasCabin
0,0,22.000000,NaN,S,0,0
1,1,38.000000,C85,C,1,1
2,1,26.000000,NaN,S,1,0
3,1,35.000000,C123,S,1,1
4,0,35.000000,NaN,S,0,0
5,0,29.699118,NaN,Q,0,0


In [22]:
# Let's drop ['Cabin'] now, we don't need it anymore
df_cleaned = df_cleaned.drop(['Cabin'], axis = 1)
df_cleaned.head(6)

,Gender,Age,Embarked,Survived,HasCabin
0,0,22.000000,S,0,0
1,1,38.000000,C,1,1
2,1,26.000000,S,1,0
3,1,35.000000,S,1,1
4,0,35.000000,S,0,0
5,0,29.699118,Q,0,0


### 5. Remove 'Embarked' rows with NaN values
There are only a few, let's identify them.

In [23]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gender    891 non-null    int64  
 1   Age       891 non-null    float64
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    int64  
 4   HasCabin  891 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 34.9+ KB


In [24]:
# What values exist in the 'Embarked' column?
df_cleaned['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [25]:
# How many rows in Embarked are NaN?
df_cleaned['Embarked'].isnull().sum()

2

In [27]:
# Look at just the rows with Embarked = NaN
df_cleaned[df_cleaned['Embarked'].isnull()]

,Gender,Age,Embarked,Survived,HasCabin
61,1,38.0,NaN,1,1
829,1,62.0,NaN,1,1


In [28]:
# Remove any rows with NaNs. This does it for the entire dataframe, not just the Embarked column
df_cleaned.dropna(inplace = True) # inplace = True makes this a permanent change
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gender    889 non-null    int64  
 1   Age       889 non-null    float64
 2   Embarked  889 non-null    object 
 3   Survived  889 non-null    int64  
 4   HasCabin  889 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 41.7+ KB


### 6. Convert 'Embarked' column to a set of 3 dummy variable columns

A good way to represent categorical data is with binary numbers in columns for each category.<P>
    
For example, Embarked has 3 categories: S, C, Q, representing the city the passenger embarked. We'll perform something called, one hot encoding <P>
    
One hot encoding is a process by which categorical variables are converted into a form that help ML algorithms to do a better job of prediction. <P>
    
You can read more about it here:<P>
- https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f    

Sometimes, they are called dummy variables too. There are several ways to do this, but we'll use the pandas dataframe method: <P>
- https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [29]:
# Use the pandas get_dummies() on the Embarked columns
dummy_df = pd.get_dummies(df_cleaned['Embarked']) # Creates a new dataframe
dummy_df.head()

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [30]:
# Add dummy_df to the end df_cleaned
df_cleaned = pd.concat([df_cleaned, dummy_df], axis=1) # Concat function
df_cleaned.head()

,Gender,Age,Embarked,Survived,HasCabin,C,Q,S
0,0,22.0,S,0,0,0,0,1
1,1,38.0,C,1,1,1,0,0
2,1,26.0,S,1,0,0,0,1
3,1,35.0,S,1,1,0,0,1
4,0,35.0,S,0,0,0,0,1


### 7. Save your cleaned data to a pickle file on S3

In [31]:
# Drop Embarked and reorder columns
titanic_clean = df_cleaned[['Gender','Age','HasCabin','C','Q','S','Survived']]
titanic_clean.head()

,Gender,Age,HasCabin,C,Q,S,Survived
0,0,22.0,0,0,0,1,0
1,1,38.0,1,1,0,0,1
2,1,26.0,0,0,0,1,1
3,1,35.0,1,0,0,1,1
4,0,35.0,0,0,0,1,0


In [32]:
# Save to the shared S3 bucket under your own folder:
#
destination_bucket = 'machinelearning-shared' # Same as before
destination_key = 'data/kcolvin/titanic_clean.pkl' # Customize for your use
#
# let's use a pickle file
#
# Create a new, empty BytesIO object
pickle_buffer = BytesIO()
#
# Load the DataFrame into the pickle object
titanic_clean.to_pickle(pickle_buffer)
#
pickle_buffer.seek(0) # This sets the stream postion to the start. Needed to send the data to the S3 bucket
#
# Use put_object() to upload the csv_buffer to a csv file in the S3 location
response = s3.put_object(Bucket=destination_bucket, Body=pickle_buffer.getvalue(), Key=destination_key)
response

{'ResponseMetadata': {'RequestId': 'ESCYWMGWZV2DRMAV',
  'HostId': '++Bpb+5Y0wu0w3XHcQCS8F9+hVySffWRy7N4bN8A7xuG/YRbZpRqZQQ8AKDiG6Jp9qT5NIjrrSM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '++Bpb+5Y0wu0w3XHcQCS8F9+hVySffWRy7N4bN8A7xuG/YRbZpRqZQQ8AKDiG6Jp9qT5NIjrrSM=',
   'x-amz-request-id': 'ESCYWMGWZV2DRMAV',
   'date': 'Fri, 19 Aug 2022 16:11:05 GMT',
   'etag': '"39de5ded3c4d49ebc5fb6e30bb4ddf87"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"39de5ded3c4d49ebc5fb6e30bb4ddf87"'}

In [33]:
# Verify your file uploaded
#
response = s3.list_objects(Bucket = destination_bucket)
#
# Parse though the response
#
my_folder = 'kcolvin'
#
for object in response['Contents']:
    if my_folder in object['Key']:
        print(object['Key'])

data/kcolvin/abalone_clean.pkl
data/kcolvin/boston_summary.csv
data/kcolvin/boston_summary.pkl
data/kcolvin/diabetes/X.pkl
data/kcolvin/diabetes/y.pkl
data/kcolvin/titanic_clean.pkl
